# Titanic for beginners
it is basic introduction to Kaggle.

## Workflow
1. Import Necessary Libraries
2. Acquire training and testing data.
3. Analyze, Visualize data
  1. Outlets (errors or possibly innacurate values) ?
  2. Create new feature?
4. Clearning data
5. Choosing the Best Model
6. Creating Submission File

## 1. Import Necessary Libraries

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
import subprocess

%matplotlib inline

## 2. Acquire training and testing data

In [2]:
train_df = pd.read_csv('../input/train.csv')
test_df = pd.read_csv('../input/test.csv')
combine = [train_df, test_df]

## 3. Analyze, Visualize data
### Take a look on data

In [3]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
train_df.tail()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.00,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.00,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q


### Analyze
- **Which features are categorical?** - Survived, Sex, and Embarked. Ordinal: Pclass.
- **Which features are numerical?** - Age, Fare. Discrete: SibSp, Parch.
- **Which features are mixed data types?** - Ticket is a mix of numeric and alphanumeric data types. Cabin is alphanumeric.
- **Which features may contain errors or typos?** - Name feature may contain errors or typos as there are several ways used to describe a name including titles, round brackets, and quotes used for alternative or short names.
- **Which features contain blank, null or empty values?** - Cabin > Age > Embarked features contain a number of null values in that order for the training dataset. Cabin > Age are incomplete in case of test dataset.
- **What are the data types for various features?** - Seven features are integer or floats. Six in case of test dataset. Five features are strings (object).
- **What is the distribution of numerical feature values across the samples?**
  - Total samples are 891 or 40\% of the actual number of passengers on board the Titanic (2,224).
  - Survived is a categorical feature with 0 or 1 values.
  - Around 38\% samples survived representative of the actual survival rate at 32%.
  - Most passengers `(> 75\%)` did not travel with parents or children.
  - Nearly 30\% of the passengers had siblings and/or spouse aboard.
  - Fares varied significantly with few passengers `(<1\%)` paying as high as $512.
  - Few elderly passengers (<1\%) within age range 65-80.
  - TODO: *distribution of age and distribution survived index by age*
  - TODO: *what is survived index of single persons (without children, parents, siblings or spouse)?*
  - TODO: *what is distribution of age of single persons?*
  - TODO: *what is survived index of not single persons which has more stronger relative (wife and husband, child vs parent and etc)*
- **What is the distribution of categorical features?**
  - Names are unique across the dataset (count=unique=891)
  - Sex variable as two possible values with `65\%` male (top=male, freq=577/count=891).
  - Cabin values have several dupicates across samples. Alternatively several passengers shared a cabin.
  - Embarked takes three possible values. S port used by most passengers (top=S)
  - Ticket feature has high ratio (22\%) of duplicate values (unique=681).
  

In [5]:
print('# features:')
print(train_df.columns.values)
print('_'*40)
print('# data types:')
train_df.info()
print('_'*40)
test_df.info()

# features:
['PassengerId' 'Survived' 'Pclass' 'Name' 'Sex' 'Age' 'SibSp' 'Parch'
 'Ticket' 'Fare' 'Cabin' 'Embarked']
________________________________________
# data types:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB
________________________________________
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null

In [6]:
# numberical features
train_df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [7]:
# categorical featuers
train_df.describe(include=['O'])

,Name,Sex,Ticket,Cabin,Embarked
count,891,891,891,204,889
unique,891,2,681,147,3
top,"Moor, Master. Meier",male,1601,C23 C25 C27,S
freq,1,577,7,4,644
